<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# To Check Solar's ability

In [19]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [20]:
# Additional Contents : https://wikidocs.net/253106 

In [1]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/path.txt", "r")
data_path = file.read()
file.close()

In [2]:
# read test_samples.csv file

import pandas as pd
import os

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

prompts, answers = read_data(os.path.join(data_path, 'mmlu_pro.csv'))

## 4. Prompt engineering

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    ---
    Answer th Question. The answer format is as follow.

    ---
    Question: "What is the embryological origin of the hyoid bone?,
    (A) :The first pharyngeal arch
    (B) The first and second pharyngeal arches
    (C) The second pharyngeal arch
    (D) The second and third pharyngeal arches

    Answer: (D) The second and third pharyngeal arches
    ---
    
    ---
    Question: {question}
    Answer: 
    ---

    '''

)
chain = prompt_template | llm

responses = []

for prompt in prompts[::100]:
    response = chain.invoke({"question": prompt})
    responses.append(response.content)

KeyboardInterrupt: 

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    Please provide most correct answer. Let's think step by step.
    
    The Answer format is as follow (do not explain) : 
    Answer: (D) keyword.
    ---
    
    Question: {question}
    Answer :
    ---
        

    '''

)
chain = prompt_template | llm

responses2 = []

for prompt in prompts[::100]:
    response = chain.invoke({"question": prompt})
    responses2.append(response.content)

In [5]:
for i in responses2:
    print(i)
    print('-')


Answer: (G) 2.
-
Answer: (C) $110.35
-
Answer: (G) A theoretical model.
-
Answer: (D) 6.
-
To find the formula for the rate of change of the total cost with respect to time, we need to differentiate the cost function C(n) with respect to time t.

Given C(n) = 0.2n^2 + n + 900, we need to find the derivative of this function with respect to t.

First, we need to find the expression for n in terms of t. Since t^2 + 100t Caped Ghouls can be produced in t hours, we can express n as:

n = t^2 + 100t

Now, we can substitute this expression for n into the cost function C(n) and differentiate with respect to t.

C(t) = 0.2(t^2 + 100t)^2 + t^2 + 100t + 900

To find the derivative of C(t) with respect to t, we can use the chain rule and the power rule for differentiation.

C'(t) = 0.4(t^2 + 100t)(2t + 100) + 2t + 100

Simplifying this expression, we get:

C'(t) = 0.4(2t^3 + 200t^2 + 10000t) + 2t + 100

C'(t) = 0.8t^3 + 800t^2 + 4000t + 2t + 100

Therefore, the formula for the rate of change of t

In [6]:
answers = answers[::100]

## Check Accuracy

In [7]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [8]:
# print accuracy

cnt = 0

for answer, response in zip(answers, responses2):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/len(answers))*100}%") # 지금 100개씩 test 중임


# print(f"acc: {(cnt/len(answers))*100}%")

----------
Answer: (G) 2.
generated answer: G, answer: (A)
----------
Answer: (C) $110.35
generated answer: C, answer: (E)
----------
Answer: (G) A theoretical model.
generated answer: A, answer: (J)
----------
Answer: (D) 6.
generated answer: D, answer: (H)
----------
To find the formula for the rate of change of the total cost with respect to time, we need to differentiate the cost function C(n) with respect to time t.

Given C(n) = 0.2n^2 + n + 900, we need to find the derivative of this function with respect to t.

First, we need to find the expression for n in terms of t. Since t^2 + 100t Caped Ghouls can be produced in t hours, we can express n as:

n = t^2 + 100t

Now, we can substitute this expression for n into the cost function C(n) and differentiate with respect to t.

C(t) = 0.2(t^2 + 100t)^2 + t^2 + 100t + 900

To find the derivative of C(t) with respect to t, we can use the chain rule and the power rule for differentiation.

C'(t) = 0.4(t^2 + 100t)(2t + 100) + 2t + 100

S